In [1]:
%reset
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import defmod as dm
import multimodule_usefulfunctions as mm 
import hamiltonian_multishape as ham
import kernels as ker
import model_MultiShapeCirclesTranslation as model

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
## define reference and template shape
r = [3,2]
origin1 = [[-3,0], [6,6]]
origin2 = [[0,3], [0, -3]]
nb_points = [10,7]

source = mm.multipleCircles(origin1, r ,nb_points)
target = mm.multipleCircles(origin2, r ,nb_points)

In [3]:
## Define Deformation Modules
dim = 2
nb_shapes = 2

# Module 1: Translation of first circle
sigma1 = 10
#trans1 = dm.deformationmodules.Translations(dim, 1, sigma1)
trans1 = dm.deformationmodules.GlobalTranslation(dim, nb_points[0], sigma1)


# Module 2: Translation of second circle
sigma2 = 10
#trans2 = dm.deformationmodules.Translations(dim,1, sigma2)
trans2 = dm.deformationmodules.GlobalTranslation(dim, nb_points[1], sigma2)


# Module 3: Background Module
sigma3 = 1.5
nb_pts = nb_points[0] + nb_points[1]
background = dm.deformationmodules.Translations(dim, nb_pts, sigma3)


module_list = [trans1, trans2, background]
dim_controls = dim*(1+1+nb_pts)
gd_list = [source[0], source[1], torch.cat([source[0], source[1]],0)]

In [4]:
target_list = [*target, torch.cat([target[0], target[1]])]

In [5]:
sigma_list = [sigma1, sigma2, sigma3]
z = [mm.computeCenter(source[0]), mm.computeCenter(source[1])]

#initialize gd and momentum
mom_tmp0 = np.zeros([nb_points[0], dim])
mom_tmp1 = np.zeros([nb_points[1], dim])
gd0_list = [source[0], source[1], torch.cat([source[0], source[1]],0)]
mom0_list = [torch.tensor(mom_tmp0.copy(), requires_grad=True, dtype=torch.float32), torch.tensor(mom_tmp1.copy(),requires_grad=True, dtype=torch.float32), torch.tensor(np.zeros(gd_list[2].shape),requires_grad=True, dtype=torch.float32)]

# Constraints (identity)
Constr1 = torch.cat([torch.eye(nb_points[0]), torch.zeros([nb_points[0], nb_points[1]]), -torch.eye(nb_points[0]), torch.zeros([nb_points[0], nb_points[1]])], 1)
Constr2 = torch.cat([torch.zeros([nb_points[1], nb_points[0]]), torch.eye(nb_points[1]), torch.zeros([nb_points[1], nb_points[0]]), -torch.eye(nb_points[1]),], 1)
Constr = [Constr1, Constr2]

# reduced Hamiltonian 
dim_control = [[1,2], [1,2], [nb_pts,2]]
H = ham.Hamiltonian_Multishape(module_list, dim_control)

In [6]:
# set parameters
reg_param = 10**-2 # regularisation parameter

gd0_tensor = torch.cat([gd0_list[0], gd0_list[1], gd0_list[2]],0)
mom0_tensor = torch.cat([mom0_list[0], mom0_list[1], mom0_list[2]],0)
    
EnergyFunctional = model.EnergyFunctional(module_list, H, Constr, target_list, sigma_list, dim, reg_param)
gradE = torch.autograd.grad(EnergyFunctional.energy_tensor(gd0_tensor, mom0_tensor), mom0_tensor)
    
# do registration
X = [gd0_tensor, mom0_tensor]
X = model.gradientdescent(EnergyFunctional.energy_tensor, EnergyFunctional.gradE, X)

 iter : 0  ,total energy: 73.64154815673828
 iter : 1  ,total energy: 71.89247131347656
 iter : 2  ,total energy: 70.15327453613281
 iter : 3  ,total energy: 68.42404174804688
 iter : 4  ,total energy: 66.70474243164062
 iter : 5  ,total energy: 64.99543762207031
 iter : 6  ,total energy: 63.29615020751953
 iter : 7  ,total energy: 61.60702133178711
 iter : 8  ,total energy: 59.92836380004883
 iter : 9  ,total energy: 58.26079559326172
 iter : 10  ,total energy: 56.60561752319336
 iter : 11  ,total energy: 54.96532440185547
 iter : 12  ,total energy: 53.344459533691406
 iter : 13  ,total energy: 51.75092697143555
 iter : 14  ,total energy: 50.197914123535156
 iter : 15  ,total energy: 48.70707321166992
 iter : 16  ,total energy: 47.31349182128906
 iter : 17  ,total energy: 46.07218933105469
 iter : 18  ,total energy: 45.05960464477539
 iter : 19  ,total energy: 44.34354019165039
 iter : 20  ,total energy: 43.88632583618164
 iter : 21  ,total energy: 43.498321533203125
 iter : 22  ,tota

In [7]:
# check if autograd is correct

#torch.autograd.gradcheck(EnergyFunctional.energy_tensor, (gd0_tensor, mom0_tensor), eps=1e-2, atol=1e-2, raise_exception=True)


In [8]:
#for i in range(len(control)):
#    z_list = [mm.computeCenter(gd[i][0]), mm.computeCenter(gd[i][1]), gd[i][2]]
#    print(EnergyFunctional.cost(z_list, control[i]))
#control
# the control is constant over time, does that make sense?

In [8]:
# compute final deformation
gd0_final = mm.tensor2list(X[0], [nb_points[0], nb_points[1], nb_points[0]+nb_points[1]])
mom0_final = mm.tensor2list(X[1], [nb_points[0], nb_points[1], nb_points[0]+nb_points[1]])

gd_t, mom_t, controls_t = mm.shootMultishape(gd0_final, mom0_final, H, Constr, sigma_list, dim, n=10)


In [9]:
nb_modules = 2
N = len(controls_t)
center_t = torch.zeros( N, nb_modules, dim)
for t in range(N):
    for m in range(2):
        center_t[t, m] = mm.computeCenter(gd_t[t][m])

In [24]:
import matplotlib.cm as cm

xmin = -10
xmax = 10
ymin = -10
ymax = 10

# Geometrical descriptors at initial time
fig = plt.figure()
plt.subplot(1,2,1)  
plt.scatter(source[0][:, 0].detach().numpy(), source[0][:, 1].detach().numpy(), c='red')
plt.scatter(source[1][:, 0].detach().numpy(), source[1][:, 1].detach().numpy(), c='blue')
plt.axis('equal')
plt.axis([xmin, xmax, ymin, ymax])

# target
plt.subplot(1,2,2)  
plt.scatter(target[0][:, 0].detach().numpy(), target[0][:, 1].detach().numpy(), c='red')
plt.scatter(target[1][:, 0].detach().numpy(), target[1][:, 1].detach().numpy(), c='blue')
plt.plot(source[0][:, 0].detach().numpy(), source[0][:, 1].detach().numpy(), 'xr')
plt.plot(source[1][:, 0].detach().numpy(), source[1][:, 1].detach().numpy(), 'xb')

plt.axis('equal')
plt.axis([xmin, xmax, ymin, ymax])


# Geometrical Descriptors transported over time by shooting
fig2 = plt.figure()  
colors = cm.rainbow(np.linspace(0, 1, len(gd_t)))
for i,c in zip(range(len(gd_t)), colors):
    plt.scatter(gd_t[i][0][:,0].detach().numpy(), gd_t[i][0][:, 1].detach().numpy(), color=c)
    plt.scatter(gd_t[i][1][:,0].detach().numpy(), gd_t[i][1][:, 1].detach().numpy(), color=c)
plt.axis('equal')
plt.axis([xmin, xmax, ymin, ymax])


# controls over time
plt.figure()  
for i in range(len(controls_t)):
    mm.plotControl(controls_t[i], [center_t[i][0],center_t[i][1]])
plt.axis([xmin, xmax, ymin, ymax])
plt.axis('equal')


# target
fig4 = plt.figure()  
plt.scatter(target[0][:, 0].detach().numpy(), target[0][:, 1].detach().numpy(), c='red')
plt.scatter(target[1][:, 0].detach().numpy(), target[1][:, 1].detach().numpy(), c='blue')
i=-1
plt.plot(gd_t[i][0][:,0].detach().numpy(), gd_t[i][0][:, 1].detach().numpy(), 'xr')
plt.plot(gd_t[i][1][:,0].detach().numpy(), gd_t[i][1][:, 1].detach().numpy(), 'xb')
plt.axis('equal')
plt.gca().set_adjustable("box")
plt.axis([xmin, xmax, ymin, ymax])


plt.show()

In [10]:
controls1_t = []
controls2_t = []
controls3_t = []
gd3_t = []

for t in range(len(controls_t)):
    controls1_t.append(controls_t[t][0])
    controls2_t.append(controls_t[t][1])
    controls3_t.append(controls_t[t][2])
    gd3_t.append(gd_t[t][2])

In [11]:
nx, ny = 50,50
sx, sy = 20, 20
x, y = torch.meshgrid([torch.arange(0, nx), torch.arange(0, ny)])
x = sx*(x.type(torch.FloatTensor)/nx - 0.5)
y = sy*(y.type(torch.FloatTensor)/ny - 0.5)

gridpoints = dm.usefulfunctions.grid2vec(x, y).type(torch.FloatTensor)


phi1 = mm.largeDeformation(trans1, center_t[:,0,:], controls1_t, gridpoints)
x1, y1 = dm.usefulfunctions.vec2grid(phi1, nx, ny)

phi2 = mm.largeDeformation(trans2, center_t[:,1,:], controls2_t, gridpoints)
x2, y2 = dm.usefulfunctions.vec2grid(phi2, nx, ny)

phi3 = mm.largeDeformation(background, gd3_t, controls3_t, gridpoints)
x3, y3 = dm.usefulfunctions.vec2grid(phi3, nx, ny)

# Final Deformation of the Multishape module
 # using the deformatation corresponding to the label of each gridpoint
%matplotlib qt
grid = [x,y]
phi = [[x1,y1], [x2,y2], [x3,y3]]
label = mm.pointInCircles(x,y,z,r)

mm.plot_MultiGrid(phi, grid, label)
i=-1
plt.plot(gd_t[i][0][:,0].detach().numpy(), gd_t[i][0][:, 1].detach().numpy(), 'xr')
plt.plot(gd_t[i][1][:,0].detach().numpy(), gd_t[i][1][:, 1].detach().numpy(), 'xb')

plt.show()